### sklearn.ensemble.StackingClassifier
### sklearn.ensemble.StackingRegressor
- 여러 학습기에서 예측한 예측값으로 다시 학습데이터를 만들어 일반화된 최종 모델을 구성하는 방법
- 데이터셋이 아닌 예측값으로 예측을 한다는 아이디어가 핵심적인 차이

#### 주요 Hyperparameter
- n_estimators : 부트스트랩 데이터셋 수

##### BaggingClassifier(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)
##### BaggingRegressor(estimators, final_estimator=None, *, cv=None, n_jobs=None, passthrough=False, verbose=0)

- Voting 방법과 유사하게 여러 개별 알고리즘으로 estimators 설정

# 분석 코드 - Classification

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, StackingClassifier, StackingRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score

df = pd.read_csv('../input/big-data-certification-study/breast-cancer-wisconsin.csv', encoding='utf-8')
df.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
# 데이터 분리
X=df.drop(columns=['code','Class'])
y=df[['Class']]
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42)

In [3]:
# 정규화
scaler=MinMaxScaler()
scaler.fit(X_train)
mm_X_train=scaler.transform(X_train)
mm_X_test=scaler.transform(X_test)

In [4]:
# 모델 적용
em=[('rf', RandomForestClassifier(n_estimators=10,random_state=42)),
    ('svm',SVC(random_state=42))]
model=StackingClassifier(estimators=em, final_estimator=LogisticRegression())
model.fit(mm_X_train, y_train)
y_pred=model.predict(mm_X_test)
accuracy_score(y_test,y_pred)

0.9649122807017544

In [5]:
# 혼동행렬, 분류예측 보고서
cm_train=confusion_matrix(y_test,y_pred)
cfr_train=classification_report(y_test,y_pred)
print('혼동행렬 :\n',cm_train,'\n',
      '분류예측 보고서 :\n',cfr_train,'\n\n')

혼동행렬 :
 [[106   5]
 [  1  59]] 
 분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
           1       0.92      0.98      0.95        60

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171
 




# 분석 코드 - Regression

In [6]:
df2=pd.read_csv('../input/big-data-certification-study/house_price.csv', encoding='utf-8')
df2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [7]:
# 데이터 분리
X=df2.drop(columns=['house_value'])
y=df2[['house_value']]
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [8]:
# 정규화
scale=MinMaxScaler()
scale.fit(X_train)
ms_X_train=scale.transform(X_train)
ms_X_test=scale.transform(X_test)

In [9]:
# 모델 적용
ems=[('lr', LinearRegression()),
     ('knn',KNeighborsRegressor())]
model=StackingRegressor(estimators=ems,
                        final_estimator=RandomForestRegressor(n_estimators=10,
                                                               random_state=42))
model.fit(ms_X_train, y_train)

pred_x=model.predict(ms_X_train)
model.score(ms_X_train, y_train)

0.6024482972648368

In [10]:
pred_y=model.predict(ms_X_test)
model.score(ms_X_test, y_test)

0.5311259081832682

In [11]:
# RMSE
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x))
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y))
print('Train RMSE :', round(rmse_train),
      '\nTest RMSE :', round(rmse_test))

Train RMSE : 60179 
Test RMSE : 65462


- 스태킹 모델은 각 알고리즘을 수행한 후 좋은 결과를 보이는 알고리즘 및 하이퍼파라미터로 쌓아야 좋은 모델을 만들 수 있는데, 이런 과정이 없다면 현 실습과 같이 개별 알고리즘 실행결과보다 저조한 성능을 보일 수 있음
- 스태킹 모델을 구성하는 개별 알고리즘의 최적화 및 모델의 순서는 스태킹 모델에 중요한 요소